# Word2Vec Embeddings

In [1]:
import pandas as pd

from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

Loading the data for which we will train word embeddings

In [2]:
col_list = ['CleanedTweet']
data = pd.DataFrame()
data = pd.read_csv('Datasets/cleaned_vaccine_tweets.csv', encoding = 'utf-8', usecols=col_list, dtype = str)
data['CleanedTweet'] = data['CleanedTweet'].apply(str)
data.head()

,CleanedTweet
0,looks like study ga device vaccine treatment l...
1,uga team say vaccine coronavirus
2,draw line wa riding followers vote jess vote a...
3,make carona vaccine make mandatory child get d...
4,exciting news second vaccine trial site


In [3]:
#create empty list
tweet_data_list = []

indv_lines = data['CleanedTweet'].values.tolist()
for line in indv_lines:
    
    #create word tokens
    rem_tok_punc = RegexpTokenizer(r'\w+')
    tokens = rem_tok_punc.tokenize(line)
    
    #append words in the tweet_data_list list
    tweet_data_list.append(tokens)

In [4]:
print(len(tweet_data_list))

195090


Training the Word2Vec model

In [5]:
# Dimension of the word embedding
embed_dim = 100

# Train Word2Vec model
model = Word2Vec(sentences = tweet_data_list, size = embed_dim, workers = 4, min_count = 1)

In [6]:
#Save word embedding model to txt file
model_file = 'Datasets/Word2Vec_embedding.txt'
model.wv.save_word2vec_format(model_file, binary=False)

# Saving the Word2Vec file
model.save("Datasets/Word2Vec.model")

In order to save time, you can load Word2Vec model directly, that was trained in the above 2 cells, by running the following cell instead:

In [18]:
# The Word2Vec model is saved in the 'Datasets' folder, under the name 'Word2Vec.model'
Word2Vec_model = 'Datasets/Word2Vec.model'

# Load trained Word2Vec model
model = Word2Vec.load(Word2Vec_model)

## Exploring the vectors in the Word2Vec embeddings

In [9]:
# Finding similar words
model.wv.most_similar('sad')

[('awful', 0.7260233163833618),
 ('crazy', 0.7105540633201599),
 ('realize', 0.7098122835159302),
 ('hate', 0.7073237895965576),
 ('wake', 0.701075553894043),
 ('scary', 0.6832720041275024),
 ('thinking', 0.6807975769042969),
 ('sadly', 0.6791499853134155),
 ('unfortunately', 0.6774884462356567),
 ('omg', 0.6770204305648804)]

In [11]:
#Performing some mathematics on word vectors queen + man - woman = ?
model.wv.most_similar_cosmul(positive=['woman','boy'], negative=['girl'])

[('preditors', 0.8686777353286743),
 ('men', 0.8650608062744141),
 ('betty', 0.8446263074874878),
 ('muslim', 0.8435612320899963),
 ('hispanics', 0.840418815612793),
 ('muthafuckaing', 0.8388912081718445),
 ('megalomaniac', 0.8363116979598999),
 ('british', 0.8343818187713623),
 ('kettle', 0.8343255519866943),
 ('oxfam', 0.8302932381629944)]

In [12]:
#Finding the odd word out from the list of words given
print(model.wv.doesnt_match("apple orange banana car".split()))

car


/Users/victoriaadcock/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


## Visualising the word embedding vectors

In [13]:
# Importing bokeh libraries for showing how words of similar context are grouped together
import bokeh.plotting as bp

from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [14]:
VocabKeys = []
for key in model.wv.vocab.keys():
    VocabKeys.append(key)


#Defining the chart
output_notebook()
plot_chart = bp.figure(plot_width=700, plot_height=600, title="A Plot of 5000 Word Vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

#Extracting the list of word vectors, limiting to 5000, each is of 200 dimensions
word_vectors = [model[w] for w in VocabKeys[:5000]]

Loading BokehJS ...

/Users/victoriaadcock/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


## Reducing dimensionality by converting the vectors to 2d vectors

## TSNE

In [15]:
from sklearn.manifold import TSNE

In [16]:
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# Storing data in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = VocabKeys[:5000]

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.001s...
[t-SNE] Computed neighbors for 5000 samples in 0.591s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.133977
[t-SNE] KL divergence after 250 iterations with early exaggeration: 80.111229
[t-SNE] KL divergence after 1000 iterations: 2.209914


In [19]:
# Corresponding word appears when you hover on the data point.
plot_chart.scatter(x='x', y='y', source=tsne_df)
hover = plot_chart.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_chart)

## PCA

In [20]:
from sklearn.decomposition import PCA

In [21]:
pca_model = PCA(n_components=2, random_state=0)
pca_w2v = pca_model.fit_transform(word_vectors)

# Storing data in a dataframe
pca_df = pd.DataFrame(pca_w2v, columns=['x', 'y'])
pca_df['words'] = VocabKeys[:5000]

In [22]:
# Corresponding word appears when you hover on the data point.
plot_chart.scatter(x='x', y='y', source=pca_df)
hover = plot_chart.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_chart)